<a href="https://colab.research.google.com/github/vinhhccode/UCTVex-and-DEA/blob/main/DEA_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 38.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from google.colab import files
from pulp import LpMinimize, LpProblem, LpVariable, lpSum
import warnings
warnings.filterwarnings('ignore')

In [3]:
import io
# Tải tệp lên Google Colab
uploaded = files.upload(r"DEA.xlsx")

# Lấy tên tệp từ dictionary 'uploaded' và sử dụng io.BytesIO để đọc tệp
for filename in uploaded.keys():
    df = pd.read_excel(io.BytesIO(uploaded[filename]))

# Kiểm tra dữ liệu
df

Saving Do dea phitc.xlsx to DEA.xlsx/Do dea phitc.xlsx


,CP,year,TỔNG TÀI SẢN,GVHB,CPBH,CPQLDN,CPTC,Net_revenue,LNST,DT hoạt động TC
0,AAA,2014,1421650734913,-1377898703125,-78937784265,-35977045351,-25896787122,1560643609091,47780875107,8331971297
1,AAA,2015,1954764678040,-1424741156735,-75230050612,-41608410264,-49991756319,1614548947901,40548813597,22394967435
2,AAA,2016,3077615978737,-1836734397150,-56882135462,-62622750100,-53927481929,2143769808850,142925017617,30198848862
3,AAA,2017,4576157128799,-3519223197973,-105724321494,-79322555685,-95052847758,4069608303141,263364105956,55775260107
4,AAA,2018,7529166722403,-7338636540198,-156562809984,-123618312196,-208587237486,8011572613389,212148913561,61517601742
...,...,...,...,...,...,...,...,...,...,...
2845,VTV,2019,1133449616013,-1914382094408,-154914837811,-38386846344,-50839636300,2169602607962,12914552660,1018589963
2846,VTV,2020,941212261608,-1342593703340,-90919922977,-33748033784,-28222822713,1501206609499,2024037235,1696500090
2847,VTV,2021,946598804603,-1810027249289,-133290018652,-45174917481,-15206199314,2018167235754,14034309968,540403115
2848,VTV,2022,1465040250216,-4034615925731,-317099830541,-55954791736,-27208416772,4449769735079,19324279626,41634858


In [5]:
# Xử lý dữ liệu
# Chuyển giá trị âm thành dương
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].abs()

# Thay thế NaN bằng 0
df = df.fillna(0)

In [6]:
# Tính trung bình hoặc tổng hợp dữ liệu cho mỗi doanh nghiệp qua toàn bộ giai đoạn
df_grouped = df.groupby('CP').mean().reset_index()

# Định nghĩa inputs và outputs
inputs = ['TỔNG TÀI SẢN', 'GVHB', 'CPBH', 'CPQLDN', 'CPTC']  # Tổng hợp các chi phí làm đầu vào
outputs = ['Net_revenue', 'LNST', 'DT hoạt động TC']  # Đầu ra

# Tạo danh sách DMUs (mỗi DMU là CP, không phân theo năm)
dmus = df_grouped['CP'].tolist()

# Chuẩn bị dữ liệu đầu vào và đầu ra
input_data = df[inputs].values
output_data = df[outputs].values

In [10]:
# --- Tính DEA trung bình cả giai đoạn (như code ban đầu) ---
df_grouped_overall = df.groupby('CP').mean().reset_index()
inputs = ['TỔNG TÀI SẢN', 'GVHB', 'CPBH', 'CPQLDN', 'CPTC']
outputs = ['Net_revenue', 'LNST', 'DT hoạt động TC']
dmus_overall = df_grouped_overall['CP'].tolist()
input_data_overall = df_grouped_overall[inputs].values
output_data_overall = df_grouped_overall[outputs].values

# --- Tính DEA theo từng năm cho mỗi mã cổ phiếu ---
df_grouped_yearly = df.groupby(['CP', 'year']).mean().reset_index()
df_grouped_yearly['DMU'] = df_grouped_yearly['CP'] + '_' + df_grouped_yearly['year'].astype(str)
dmus_yearly = df_grouped_yearly['DMU'].tolist()
input_data_yearly = df_grouped_yearly[inputs].values
output_data_yearly = df_grouped_yearly[outputs].values

# Hàm tính DEA (Input-oriented, CRS model - CRSTE)
def calculate_dea_crs(input_data, output_data, dmus):
    efficiencies = []
    for i in range(len(dmus)):
        prob = LpProblem(f"DEA_{dmus[i]}", LpMinimize)
        theta = LpVariable(f"theta_{dmus[i]}", lowBound=0)
        lambdas = [LpVariable(f"lambda_{dmus[i]}_{j}", lowBound=0) for j in range(len(dmus))]
        prob += theta
        for k in range(input_data.shape[1]):
            prob += lpSum(lambdas[j] * input_data[j][k] for j in range(len(dmus))) <= input_data[i][k] * theta
        for k in range(output_data.shape[1]):
            prob += lpSum(lambdas[j] * output_data[j][k] for j in range(len(dmus))) >= output_data[i][k]
        prob.solve()
        efficiency = theta.value()
        efficiencies.append(efficiency if efficiency is not None else 0)
    return efficiencies

# Hàm tính DEA (Input-oriented, VRS model - VRSTE)
def calculate_dea_vrs(input_data, output_data, dmus):
    efficiencies = []
    for i in range(len(dmus)):
        prob = LpProblem(f"DEA_{dmus[i]}", LpMinimize)
        theta = LpVariable(f"theta_{dmus[i]}", lowBound=0)
        lambdas = [LpVariable(f"lambda_{dmus[i]}_{j}", lowBound=0) for j in range(len(dmus))]
        prob += theta
        for k in range(input_data.shape[1]):
            prob += lpSum(lambdas[j] * input_data[j][k] for j in range(len(dmus))) <= input_data[i][k] * theta
        for k in range(output_data.shape[1]):
            prob += lpSum(lambdas[j] * output_data[j][k] for j in range(len(dmus))) >= output_data[i][k]
        prob += lpSum(lambdas) == 1
        prob.solve()
        efficiency = theta.value()
        efficiencies.append(efficiency if efficiency is not None else 0)
    return efficiencies

# --- Tính DEA cho trung bình cả giai đoạn ---
crste_scores_overall = calculate_dea_crs(input_data_overall, output_data_overall, dmus_overall)
vrste_scores_overall = calculate_dea_vrs(input_data_overall, output_data_overall, dmus_overall)

# Tạo DataFrame kết quả cho trung bình cả giai đoạn
results_overall = pd.DataFrame({
    'DMU': dmus_overall,
    'CRSTE': crste_scores_overall,
    'VRSTE': vrste_scores_overall,
})

# --- Tính DEA theo từng năm ---
crste_scores_yearly = calculate_dea_crs(input_data_yearly, output_data_yearly, dmus_yearly)
vrste_scores_yearly = calculate_dea_vrs(input_data_yearly, output_data_yearly, dmus_yearly)

# Tạo DataFrame kết quả cho từng năm
results_yearly = pd.DataFrame({
    'DMU': dmus_yearly,
    'CP': df_grouped_yearly['CP'],
    'year': df_grouped_yearly['year'],
    'CRSTE': crste_scores_yearly,
    'VRSTE': vrste_scores_yearly,
})

# Hiển thị kết quả
print("\nKết quả DEA trung bình cho từng mã cổ phiếu trong toàn bộ giai đoạn (2014-2023):")
print(results_overall)

print("\nKết quả DEA cho từng mã cổ phiếu theo từng năm (2014-2023):")
print(results_yearly)


Kết quả DEA trung bình cho từng mã cổ phiếu trong toàn bộ giai đoạn (2014-2023):
     DMU     CRSTE     VRSTE
0    AAA  0.855687  0.876069
1    AAM  0.881708  0.898854
2    ABT  0.870999  0.873099
3    ACC  0.843917  0.846264
4    ACL  0.836351  0.836921
..   ...       ...       ...
280  VSC  0.970715  0.999945
281  VSI  0.887564  0.900334
282  VTB  0.888371  0.896780
283  VTC  0.829085  0.835563
284  VTV  0.924875  0.925693

[285 rows x 3 columns]

Kết quả DEA cho từng mã cổ phiếu theo từng năm (2014-2023):
           DMU   CP  year     CRSTE     VRSTE
0     AAA_2014  AAA  2014  0.753699  0.754109
1     AAA_2015  AAA  2015  0.705851  0.708550
2     AAA_2016  AAA  2016  0.695903  0.721842
3     AAA_2017  AAA  2017  0.744508  0.757604
4     AAA_2018  AAA  2018  0.750835  0.798088
...        ...  ...   ...       ...       ...
2845  VTV_2019  VTV  2019  0.845772  0.846066
2846  VTV_2020  VTV  2020  0.807024  0.807266
2847  VTV_2021  VTV  2021  0.846177  0.854443
2848  VTV_2022  VTV  2022